# Perturbar Series de Tiempo
### 1.- Quitar valores en %
### 2.- Agregar Outliers en %
### 3.- Agregar SNR en dB

In [1]:
import os, sys
sys.path.insert(0, '/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers')
import outliersDetection as od
import requests, pandas as pd, numpy as np
import time, json
import bokeh
from scipy import signal, fft
# %matplotlib inline
from bokeh.plotting import gridplot, figure, output_file, output_notebook, show, ColumnDataSource
from datetime import datetime
from pylab import *

None


## Valores faltantes

In [4]:
def quitarValores(ts,porcentaje):
    cambios = []
    tam = len(ts)
    elementos = round((porcentaje/100.0)*tam)
#     print("elementos",elementos)
    indices = []
    for i in range(elementos):
        k = 0
        while k == 0:
            r=np.random.randint(tam)
            if r not in indices: 
                indices.append(r)
                k = 1
#     print("FALTANTES",indices)
    for i in indices:
        ts[i] = np.nan
    return ts

## Outliers

In [5]:
def agregarOutliers(ts,porcentaje):
    cambios = []
    tam = len(ts)
    elementos = round((porcentaje/100)*tam)
#     print("elementos",elementos)
    indices = []
    for i in range(elementos):
        k = 0
        while k == 0:
            r=np.random.randint(tam)
            if r not in indices and ts[r] != np.nan: 
                indices.append(r)
                k = 1
#     print("OUTLIERS INDICES",indices)
#     media = np.mean(ts)
#     sigma = np.std(ts)*5
    for i in indices:
        ts[i] += ts[i]*2
#         print("\nts_i despues",ts[i])
#         else:
#             alpha = np.random.rand()**np.random.randint(low=2,high=5)
#             ts[i] = media*((sigma+alpha))
    return ts

## SNR

In [6]:
# x = A*np.cos(2*np.pi*1000*t) + a*np.sin(2*np.pi*2000*t) + s*np.random.normal(len(t));
def funcionSeno(fs, f, T=2):
    fs_ = fs #100KHz frecuencia de muestreo
    f_ = f #1KHz frecuencia de la senal
    t_ = np.arange(0,2*(1/f_),1/fs_)
#     print(len(t_))
    x_ = np.sin(T*np.pi*f_*t_)
    return x_, t_

def agregarSNR(x,SNR_dB,t_):
    #https://www.gaussianwaves.com/gaussianwaves/wp-content/uploads/2015/06/How_to_generate_AWGN_noise.pdf
    #Matlab / Octave tiene una función incorporada llamada -awgn () con la que se puede agregar 
    #un ruido blanco para obtener la señal deseada a ruido (SNR). 
    #El uso principal de esta función es agregar AWGN a una señal limpia (SNR infinita) para obtener una señal 
    #tamano de la serie
    L = len(x)
    #SNR a escala lineal
    SNR = np.power(10,(SNR_dB/10))
    #Calcular la energía
    Esym = np.sum(np.power(abs(x),2))/(L);
    #Encontrar la densidad espectral de ruido
    N0=Esym/SNR;
    #Desviación estándar para el Ruido
    noiseSigma =np.sqrt(N0);
    #Creando el ruido
    n = np.random.normal(0,noiseSigma,L)
#     print("SNR medido en dB",validarSNRinyectado(x, n,t_))
#     print("x\n\n",x,"\n\n\n n \n\n\n",n)
    return x + n

#Funcion para calcular el SNR antes de inyectarlo
def validarSNRinyectado(sig, noise,dt):
    Signal = np.sum(np.abs(np.fft.fft(sig)*dt)**2)/len(np.fft.fft(sig))
    Noise = np.sum(np.abs(np.fft.fft(noise)*dt)**2)/len(np.fft.fft(noise))
#     print("SNR medido lineal",Signal/Noise)
    return (10 * np.log10(Signal/Noise))

In [7]:
def plotseries(tipo,path="",ts=[]):
    # Funcion para plotear las series
    if tipo == 0:
        od.grafica2D(np.arange(len(ts)),ts,title1='TS',tipo=2)
    else:
        ts = pd.read_csv(path)
#         ts = ts.dropna()
        od.grafica2D(np.arange(len(ts)),ts['col1'],title1='TS',tipo=2)


def crearStPerturbada(nombreTs,showPlot=0):
    #nombreTs: Nombre de la serie de tiempo
    #cuantasSeries: Numero de series independientes
    niveles = [0,5,10,15,20,25]
    ###
    # print(ts[0:20])
    # Primer nivel para quitar valores
    for i in niveles:
        for j in niveles:
            for k in niveles:
#                 print("nombre",nombreTs,"_",i,"_",j,"_",k)
                ##BLOQUE PARA ABRIR Y QUITAR VALORES
                # filename = "st con ruido/seno5/ts 0.csv"
                filename = "st originales/"+nombreTs+".dat"
                ts = pd.read_csv(filename)
                ts = ts['col1']
                tamano = len(ts)
                if tamano > 20000:
                    ts = ts[0:20000]
                if i != 0:
        #             print("valores %",i)
                    ts = quitarValores(ts,i)
                
                #BLOQUE PARA AGREGAR OUTLIERS
                # valoresFaltantes(ts_aux2)
                if j != 0:
                    ts = agregarOutliers(ts,j)
                
                #BLOQUE PARA AGREGAR SNR
                # Ruta donde se va a guardar la ts
                path = "st perturbadas/"+nombreTs
                # Revisando que exista el directorio si no existe lo crea
                if not os.path.exists(path): 
                    os.mkdir(path)
                # Crea el archivo .csv donde se guardara la ts
                csv_file2 = open(path+"/"+nombreTs+'_'+str(i)+'_'+str(j)+'_'+str(k)+'.csv', "w")
#                 print("Datos faltantes",i,"% Outliers",j,"% SNR",k,"dB")
                if k != 0:
                    SNR_dB = k
                    # Usando la funcion agregarSNR para inyeectar el ruido
                    xn_ = agregarSNR(ts,SNR_dB,np.arange(len(ts)))
                    xn2 = pd.DataFrame({'col0': np.arange(len(xn_)),'col1': xn_})
                    csv_file2.write(xn2.to_csv(index=False))
                    csv_file2.close()
                    if showPlot: plotseries(0,ts=xn_)
                else:
                    xn2 = pd.DataFrame({'col0': np.arange(len(ts)),'col1': ts})
                    csv_file2.write(xn2.to_csv(index=False))
                    csv_file2.close()
                    if showPlot: plotseries(0,ts=ts)
    print("Termino")
    

In [6]:
# SC = ["henon","lorenz","rossler","duffing_oscillator","halvorsen_attractor",
#       "rucklidge_attractor","shawn_van_der_pol_oscillator","simplest_cubic_flow","simplest_linear_flow"]
# SC = ["seno"]
# SC = ["chen_system"]
# for i in SC:
#     crearStPerturbada(i,showPlot=0)

In [8]:
def grafica2D(ts,titulo="Grafica",title1="Serie 1"):    
    ts["timestamp"] = pd.to_datetime(ts["timestamp"])
    ts.set_index("timestamp", inplace=True)
    
    p = bokeh.plotting.figure(title=titulo,plot_width=800, plot_height=400, x_axis_type='datetime')
    output_notebook()
    legend_it = []
#     print("x1\n",x1)
#     f.line(df.index, range(len(df.index)))
    p.line(ts.index, ts['Vav'], color="black")
    #p.circle(x1,y1, color="red")
    bokeh.plotting.show(p)

#grafica2D(x1 = ts['timestamp'], y1 = ts['kw'])

In [27]:
filename = "st perturbadas/simplest_linear_flow/simplest_linear_flow_0_0_0.csv"
ts = pd.read_csv(filename)
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_aristeomercado_10mC.csv')
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_cointzio_10mC.csv')
# ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_corrales_10mC.csv')
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_elfresno_10mC.csv')
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_lapalma_10mC.csv')
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/Original/wind_patzcuaro_10mC.csv')
#ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/snips/aristeomercado_snipC.csv')
# ts = pd.read_csv('/home/victort/Documentos/SEMESTRE 17-18 MAESTRIA/TESIS/Deteccion Outliers/michoacan stations/michoacan stations/snips/cointzio_snipC.csv')

ts = ts['col1']
od.grafica2D(np.arange(len(ts)),ts,titulo = "Flujo lineal por partes más simple",title1 = 'Variable X')
# print(filename.split("/")[1])
#od.obtenerOutliers(ts,showPlot=1)
#plotseries(1,"st perturbadas/seno/seno_0_5_0.csv")


#filename = "st perturbadas/rossler/rossler_5_0_0.csv"
#ts = pd.read_csv(filename)
#PLOTS CFE CIRCUITS
# filename = "/home/victort/MEGAsync/CodigoMIRD/Segundo preproceso nuevos modelos offline/preprocessing2/ABL04025.parquet"
#filename = "/home/victort/MEGAsync/CodigoMIRD/Data/preprocessing0/LAE04090.parquet"
# filename = "/home/victort/MEGAsync/CodigoMIRD/Data/PP/parquet/1_hour/ABL04015_.parquet"
# filename = "/home/victort/MEGAsync/CodigoMIRD/Data/fallas/parquet/HNM04060.parquet"
# ts = pd.read_parquet(filename, engine='auto')
# print(ts['timestamp'])
# # ts = ts['col1']
# grafica2D(ts)
# #plotseries(1,"st perturbadas/seno/seno_0_5_0.csv")

Loading BokehJS ...

In [9]:
# x = A*np.cos(2*np.pi*1000*t) + a*np.sin(2*np.pi*2000*t) + s*np.random.normal(len(t));
def funcionSeno(fs, f, T=2):
    fs_ = fs #100KHz frecuencia de muestreo
    f_ = f #1KHz frecuencia de la senal
    print("pasos",2*(1/f_),"hasta",1/fs_)
    t_ = np.arange(0,2*(1/f_),1/fs_)
    print("size t",len(t_))
    print("t[0:10]",t_[-1])
    x_ = np.sin(T*np.pi*f_*t_)
    return x_, t_

In [10]:
xss, tss = funcionSeno(fs=100000, f=10, T=2)
# od.grafica2D(np.arange(len(xss)),xss,titulo = "Flujo lineal por partes más simple",title1 = 'Variable X')
# np.savetxt("seno2.csv", xss, delimiter=",")

pasos 0.2 hasta 1e-05
size t 20000
t[0:10] 0.19999000000000003


In [ ]:
len(tss)

In [24]:
crearStPerturbada("seno2",showPlot=0)

Termino
